In [1]:
import time
import os
import glob
import fitz  # PyMuPDF
import base64
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

# Set up Selenium WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
prefs = {"printing.print_preview_sticky_settings.appState": '{"recentDestinations":[{"id":"Save as PDF"}]}'}
options.add_experimental_option("prefs", prefs)
options.add_argument("--kiosk-printing")  # Auto confirm print
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Load the webpage
driver.get("https://www.coursera.org/?authMode=login")  # Change to your target URL

In [2]:
driver.switch_to.window(driver.window_handles[1])

In [3]:
driver.execute_script("document.body.style.zoom='90%'")

In [6]:
driver.execute_script("return document.getElementsByClassName('rc-TunnelVisionHeader _11sxso0')[0].remove();")

In [7]:
driver.execute_script("return document.getElementsByClassName('cds-AccordionRoot-focusContainer')[0].remove();")

In [8]:
def save_pdf(driver, file_name):
    params = {'landscape': False, 'paperWidth': 12, 'paperHeight': 25}
    data = driver.execute_cdp_cmd("Page.printToPDF", params)
    with open(file_name, 'wb') as file:
        file.write(base64.b64decode(data['data']))
# Usage
output_file = "practical_quiz_1.pdf"
save_pdf(driver, output_file)

In [15]:
from spire.pdf.common import *
from spire.pdf import *

# Load a PDF document
origin_doc = PdfDocument("course_1/week_1/practical_quiz_1.pdf")

# Create a new PDF document
new_doc = PdfDocument()

# Iterate through each page of the loaded document
for i in range(origin_doc.Pages.Count):
    # Get the current page
    page = origin_doc.Pages[i]

    # Add a new page with A3 size and no margins to the new PDF document
    newPage = new_doc.Pages.Add(PdfPageSize.A4(), PdfMargins(0))

    # Create a PdfTextLayout object
    layout = PdfTextLayout()
    # Set content layout type as one page to automatically scale content according to page size
    layout.Layout = PdfLayoutType.OnePage

    # Draw the page content of the current page on the new page
    page.CreateTemplate().Draw(newPage, PointF.Empty(), layout)

# Save the new PDF document
new_doc.SaveToFile("A4.pdf")
origin_doc.Close()
new_doc.Close()

# convert_pdf_to_a4("", "output_a4.pdf")

In [19]:
!pip uninstall Spire.Pdf -y

In [22]:
!pip uninstall fitz
!pip install --upgrade --force-reinstall pymupdf

Defaulting to user installation because normal site-packages is not writeable
  Using cached pymupdf-1.25.3-cp39-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (3.4 kB)
Using cached pymupdf-1.25.3-cp39-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (20.0 MB)
  Attempting uninstall: pymupdf
    Found existing installation: PyMuPDF 1.25.3
    Uninstalling PyMuPDF-1.25.3:
      Successfully uninstalled PyMuPDF-1.25.3

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: /usr/bin/python3 -m pip install --upgrade pip


In [23]:
import fitz

In [24]:
src = fitz.open("course_1/week_1/practical_quiz_1.pdf")  # problem PDF
doc = fitz.open()
for ipage in src:
    if ipage.rect.width > ipage.rect.height:
        fmt = fitz.PaperRect("a4-l")  # landscape if input suggests
    else:
        fmt = fitz.PaperRect("a4")
    page = doc.newPage(width = fmt.width, height = fmt.height)
    page.showPDFpage(page.rect, src, ipage.number)

doc.save("reformatted.pdf")

AttributeError: module 'fitz' has no attribute 'PaperRect'

In [27]:
import fitz  # PyMuPDF

src = fitz.open("course_1/week_1/practical_quiz_1.pdf")  # Open source PDF
doc = fitz.open()  # Create a new PDF document

for ipage in src:
    rect = ipage.rect  # Get original page dimensions
    crop_rect = fitz.Rect(rect.x0 + 30, rect.y0, rect.x1 - 30, rect.y1)  # Adjust left/right padding

    if rect.width > rect.height:
        fmt = fitz.paper_size("a4-l")  # Landscape A4
    else:
        fmt = fitz.paper_size("a4")  # Portrait A4
    
    page = doc.new_page(width=fmt[0], height=fmt[1])  # Create new page
    page.show_pdf_page(page.rect, src, ipage.number, clip=crop_rect)  # Apply cropping

doc.save("reformatted2.pdf")
doc.close()
